In [6]:
#!pip install yfinance
#!pip install --upgrade yfinance
!pip install yahoo_fin

Requirement already up-to-date: yfinance in /Users/jasonmaytin/opt/anaconda3/lib/python3.8/site-packages (0.2.37)


In [12]:
from yahoo_fin.stock_info import get_data

# List of tickers for the 30 companies in the DJIA
djia_tickers = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG",
    "FB", "BRK-A", "BRK-B", "TSLA", "NVDA",
    "JPM", "JNJ", "V", "UNH", "PYPL",
    "MA", "HD", "DIS", "BAC", "CMCSA",
    "CRM", "KO", "ADBE", "NFLX", "NKE",
    "T", "PEP", "ABT", "ASML", "INTC"
]

# Fetch data for each ticker
stock_data = {}
for ticker in djia_tickers:
    try:
        data = get_data(ticker)
        stock_data[ticker] = data
    except AssertionError as e:
        print(f"No data found for {ticker}: {e}")

# Print the first few rows of the data for each ticker
for ticker in djia_tickers:
    if ticker in stock_data:
        print(f"Stock data for {ticker}:")
        print(stock_data[ticker].head())
        print("\n")

No data found for FB: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Stock data for AAPL:
                open      high       low     close  adjclose     volume ticker
1980-12-12  0.128348  0.128906  0.128348  0.128348  0.099192  469033600   AAPL
1980-12-15  0.122210  0.122210  0.121652  0.121652  0.094017  175884800   AAPL
1980-12-16  0.113281  0.113281  0.112723  0.112723  0.087117  105728000   AAPL
1980-12-17  0.115513  0.116071  0.115513  0.115513  0.089273   86441600   AAPL
1980-12-18  0.118862  0.119420  0.118862  0.118862  0.091861   73449600   AAPL


Stock data for MSFT:
                open      high       low     close  adjclose      volume  \
1986-03-13  0.088542  0.101563  0.088542  0.097222  0.060163  1031788800   
1986-03-14  0.097222  0.102431  0.097222  0.100694  0.062311   308160000   
1986-03-17  0.100694  0.103299  0.100694  0.102431  0.063386   133171200   
1986-03-18  0.102431  0.103299  0.098958

In [14]:
stock_data["AAPL"]

,open,high,low,close,adjclose,volume,ticker
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099192,469033600,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094017,175884800,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087117,105728000,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089273,86441600,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091861,73449600,AAPL
...,...,...,...,...,...,...,...
2024-03-01,179.550003,180.529999,177.380005,179.660004,179.660004,73488000,AAPL
2024-03-04,176.149994,176.899994,173.789993,175.100006,175.100006,81510100,AAPL
2024-03-05,170.759995,172.039993,169.619995,170.119995,170.119995,95132400,AAPL
2024-03-06,171.059998,171.240005,168.679993,169.119995,169.119995,68587700,AAPL
